<a href="https://colab.research.google.com/github/Christophe-Gauge/NEOWISE/blob/master/comet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comet C2020 F3 NEOWISE

Don't miss Comet NEOWISE in the evening sky now. It won't be back for 6,800 years.

Here's where to look, according to NASA.

An amazing comet that thrilled early-morning stargazers earlier this month is now visible in the evening sky, and it's a sight you won't want to miss. After all, this comet won't be back for 6,800 years, NASA says. 

Comet NEOWISE can now be seen just after sunset for observers in the Northern Hemisphere, according to NASA. (Sorry, Southern Hemisphere skywatchers, it's not visible there.) The comet made its closest approach to the sun July 3 but was only visible before dawn until now.

Find the rise and set time of this comet for your location.

This code was initially created by [Comet C2020 F3 NEOWISE in the Morning (and eventually, the evening)
](https://shallowsky.com/blog/science/astro/comet-neowise.html), you can find the original source code [here](https://github.com/akkana/scripts/blob/master/comet.py).

This interactive Python Notebook allows anyone to get the rise and set times, as well as the azimuth, to determine when teh comet will be visible. Set the parameters below and run the iPython notebook.


- Set the coordinates to your location (lat, lon and altitude in meters) as an array
- Set the Comet name (NEOWISE is 2020 F3)
- Set the time (defaults to "now")
- Set the number of days you want to see

Edit below:


In [5]:
coords = [34.398, -118.483, 500]
comet_name = '2020 F3'
numdays = 15
t = ts.now()


In [6]:
!pip install skyfield
from skyfield.api import Loader, Topos
from skyfield.data import mpc
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
from skyfield import almanac

import dateutil.parser
from datetime import timedelta

import argparse

import sys, os

load = Loader('./skyfield')
with load.open(mpc.COMET_URL) as f:
    comets = mpc.load_comets_dataframe(f)

# Index by designation for fast lookup.
comets = comets.set_index('designation', drop=False)

ts = load.timescale(builtin=True)
eph = load('de421.bsp')
sun = eph['sun']
earth = eph['earth']


def comet_by_name(namepat):
    # Exact search: row = comets.loc[cometname]

    # How to search for something containing a string in pandas:
    rows = comets[comets['designation'].str.contains(namepat)]

    # Found it!
    if len(rows) == 1:
        return rows.iloc[0]

    # No match
    if len(rows) == 0:
        return None

    # Multiple matches, so print them but return nothing.
    # pandas iterrows() returns two things but they're both the same object,
    # not an index and an object like you might expect. So ignore r.
    print("Matches from", len(comets), 'comets loaded:')

    for r, row in rows.iterrows():
        print(row['designation'])
    return None


def calc_comet(comet_df, obstime, earthcoords, numdays):
    # Generating a position.
    cometvec = sun + mpc.comet_orbit(comet_df, ts, GM_SUN)

    cometpos = earth.at(obstime).observe(cometvec)
    ra, dec, distance = cometpos.radec()
    print("RA", ra, "   DEC", dec, "   Distance", distance)

    if earthcoords:
        if len(earthcoords) > 2:
            elev = earthcoords[2]
        else:
            elev = 0
        obstopos = Topos(latitude_degrees=earthcoords[0],
                         longitude_degrees=earthcoords[1],
                         elevation_m=elev)
        print("\nObserver at",
               obstopos.latitude, "N  ", obstopos.longitude, "E  ",
              "Elevation", obstopos.elevation.m, "m")
        observer = earth + obstopos

        alt, az, distance = \
            observer.at(obstime).observe(cometvec).apparent().altaz()
        print("Altitude", alt, "     Azumuth", az, distance)

        if numdays:
            print("\nRises and sets over", numdays, "days:")
            datetime1 = obstime.utc_datetime() - timedelta(hours=2)
            t0 = ts.utc(datetime1)
            t1 = ts.utc(datetime1 + timedelta(days=numdays))

            alm = almanac.risings_and_settings(eph, cometvec, obstopos)
            t, y = almanac.find_discrete(t0, t1, alm)

            fmt = "%-10s    %-10s %-8s   %-10s %-18s"
            print(fmt % ("Date", "Rise", "Azimuth", "Set", "Az"))
            datestr = ''
            risetime = ''
            riseaz = ''
            settime = ''
            setaz = ''
            for ti, yi in zip(t, y):
                alt, az, distance = \
                    observer.at(ti).observe(cometvec).apparent().altaz()
                t = ti.utc_datetime().astimezone()
                d = t.strftime("%Y-%m-%d")
                if yi:
                    risetime =  ti.utc_datetime().astimezone() \
                                                 .strftime("%H:%M %Z")
                    riseaz = "%3d°%2d'" % az.dms()[:2]
                else:
                    settime =  ti.utc_datetime().astimezone() \
                                                .strftime("%H:%M %Z")
                    setaz = "%3d°%2d'" % az.dms()[:2]
                if not datestr:
                    datestr = d
                if d != datestr:
                    print(fmt % (datestr, risetime, riseaz, settime, setaz))
                    risetime = ''
                    riseaz = ''
                    settime = ''
                    setaz = ''
                    datestr = d

            if risetime or settime:
                print(fmt % (datestr, risetime, riseaz, settime, setaz))

comet_df = comet_by_name(comet_name)
calc_comet(comet_df, t, coords, numdays)

RA 08h 13m 47.88s    DEC +47deg 49' 32.1"    Distance 0.751345 au

Observer at 34deg 23' 52.8" N   -118deg 28' 58.8" E   Elevation 500.0 m
Altitude 52deg 27' 43.5"      Azumuth 54deg 25' 58.4" 0.751311 au

Rises and sets over 15 days:
Date          Rise       Azimuth    Set        Az                
2020-07-17    11:16 UTC   24°47'    06:20 UTC  335°23'           
2020-07-18    11:33 UTC   24°36'    06:37 UTC  335° 5'           
2020-07-19    11:53 UTC   25° 3'    06:50 UTC  334°10'           
2020-07-20    12:17 UTC   26° 6'    07:01 UTC  332°41'           
2020-07-21    12:42 UTC   27°43'    07:08 UTC  330°44'           
2020-07-22    13:09 UTC   29°48'    07:14 UTC  328°25'           
2020-07-23    13:36 UTC   32°14'    07:17 UTC  325°50'           
2020-07-24    14:02 UTC   34°54'    07:20 UTC  323° 6'           
2020-07-25    14:27 UTC   37°43'    07:21 UTC  320°16'           
2020-07-26    14:50 UTC   40°37'    07:21 UTC  317°24'           
2020-07-27    15:11 UTC   43°31'    07: